In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/KaiserHealthNews.txt'
dest_url = './run_data/kaiserhealth_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# (http:\/\/)(www)?([A-Za-z0-9\.\/]+)(\.[a-zA-Z]+)$ --> Image
# (http(s)?:\/\/(www.)?[A-Za-z0-9\/.?=_-]+(jpg|png)$)
# (http(s)?:\/\/(www.)?([A-Za-z0-9\/.?=_-]+)(jpg|png)$)
# (http(s)?:\/\/)(www.)?(youtu).([A-Za-z0-9\/.?=_-]+) --> Youtube
# (https:\/\/youtu[A-Za-z0-9\/.?=_-]+) -> youtu.be
# (http:\/\/www\.youtube.com[A-Za-z0-9\/.?=_-]+) -> youtube full link

regex_additional_links = r'(http(s)?:\/\/(www.)?[A-Za-z0-9\/.?=_-]+(jpg|png)$)'

In [3]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

scripts.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  import pandas as pd


,tweetid,date,title
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting..."


In [4]:
# -------------------------------------------------
frame.shape

(3505, 3)

In [5]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'A new sort of extracurricular activity: seeing patients. http://khne.ws/1ceRuaZ '

In [6]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
print("TITLE:\n\n ")
frame['title'] = frame['title'].map(split_txt_form_url)
# --> Get external link and strip it within title
# frame['ext_url'] = frame['title'].map(lambda row: get_url(regex_additional_links[0],row))
# frame['ext_url'] = frame['title'].map(lambda row: get_extend_link(regex_additional_links, row))
# frame['title'] = frame['title'].map(lambda row: row.replace(frame['ext_url'],'') if len(frame['ext_url']) else row)

http://khne.ws/1ceRuaZ
http://khne.ws/1NfQjsp
http://khne.ws/1Do7UrB
http://khne.ws/1y8aJwC
http://khne.ws/1HUROpc
http://khne.ws/1JowAkr
http://khne.ws/1NaiTex
http://khne.ws/1JowAkr
http://cnn.it/1yVmb9H
http://wamu.fm/1NQYhmG
http://kaiserhealthnews.org/news/khn-video-tax-deadline-meets-the-health-law/
http://kaiserhealthnews.org/news/rule-proposed-on-providing-mental-health-parity-in-medicaid-program/
http://kaiserhealthnews.org/news/battle-for-mental-health-parity-produces-mixed-results/
http://kaiserhealthnews.org/news/high-deductible-health-plans-can-ruin-finances/
http://kaiserhealthnews.org/morning-breakout/billionaires-harness-money-technology-in-pursuit-of-fountain-of-youth/
http://kaiserhealthnews.org/news/ritual-not-science-keeps-the-annual-physical-alive/
http://kaiserhealthnews.org/news/medicare-is-stingy-in-first-year-of-doctor-bonuses/
http://kaiserhealthnews.org/news/cancer-survivor-worries-about-supreme-court-ruling-on-obamacare-subsidies/
http://kaiserhealthnews.org

http://twitpic.com/c0viin
http://khne.ws/11JfEhf
http://khne.ws/HKMauB
http://khne.ws/YJjWZO
http://khne.ws/11ttUuy
http://khne.ws/14vmWIM
http://khne.ws/U986qp
http://khne.ws/11ttUuy
http://khne.ws/UJYyAc
http://khne.ws/TlD3pL
http://khne.ws/HKMauB
http://khne.ws/YlTfXj
http://khne.ws/U39URZ
http://khne.ws/14txBUp
http://khne.ws/HKMauB
http://khne.ws/Wg15mB
http://ow.ly/hgWxQ
http://khne.ws/WR6Ch8
http://khne.ws/We5uGD
http://khne.ws/Vo4hsv
http://khne.ws/10IPta2
http://khne.ws/VWyH6X
http://khne.ws/HKMauB
http://khne.ws/We5uGD
http://khne.ws/XIjvIC
http://khne.ws/YCdXWD
http://khne.ws/YC7rPD
http://www.kaiserhealthnews.org/Daily-Report.aspx
http://khne.ws/XI1r1e
http://khne.ws/HKMauB
http://khne.ws/Vj15hP
http://khne.ws/WmeqsS
http://khne.ws/VrVdIr
http://khne.ws/WlxhnD
http://khne.ws/HKMauB
http://khne.ws/WldOnC
http://huff.lv/10JusvY
http://khne.ws/Vyspdw
http://khne.ws/10IPta2
http://khne.ws/SMdLRi
http://khne.ws/HKMauB
http://khne.ws/Y1GRvp
http://khne.ws/Vg6851
http://khne.ws/W2

http://khne.ws/1k9XsXL
Ive never seen data on how many closely held businesses there are out there. http://khne.ws/1iTO6o1


http://khne.ws/1iTO6o1
ICYMI:  What exactly is a "closely held for-profit corporation"? http://khne.ws/1iTO6o1


http://khne.ws/1iTO6o1
American College of Physicians reverses guidance on pelvic exams in well-woman visits. @JennyAGold has the details. http://khne.ws/1pSZP54


http://khne.ws/1pSZP54
ICYMI: Does the #scotus have final say on contraception coverage for employees? Maybe not, reports @jrovner: http://khne.ws/1x7m634


http://khne.ws/1x7m634
Does the #scotus have final say on contraception coverage for employees? Maybe not, reports @jrovner: http://khne.ws/1x7m634 #hobbylobby


http://khne.ws/1x7m634
Will the #SCOTUS contraceptive decision have a broad or narrow impact? @JayHancock1 examines: http://khne.ws/1iTO6o1


http://khne.ws/1iTO6o1
Have #SCOTUS @HobbyLobbyStore questions? We answer: http://khne.ws/1mOaPkV


http://khne.ws/1mOaPkV
What The Hobby

Three Critical Measures Of Marketplaces Impact Could Take Several Years To Assess http://khne.ws/15Gh0fe


http://khne.ws/15Gh0fe
RT @jordanrau: How Much Premium Variation Is There Within Your State? See @khnews' Chart of 36 States. http://www.kaiserhealthnews.org/Stories/2013/September/30/premium-variation-intrastate-obamacare-marketplaces-exchanges.aspx


http://www.kaiserhealthnews.org/Stories/2013/September/30/premium-variation-intrastate-obamacare-marketplaces-exchanges.aspx
Poll Finds Most Unaware of Marketplace Opening Date. Hint: It's tomorrow. http://khne.ws/1hfduOd


http://khne.ws/1hfduOd
Latest: Battle Over Health Law Brings Gov't To Brink Of Shutdown http://khne.ws/169B11D


http://khne.ws/169B11D
Where You Live Determines How Much You Pay For Health Insurance, @Julie_Appleby and @JordanRau report: http://khne.ws/15EA7f8


http://khne.ws/15EA7f8
In 10 min., our @PhilGalewitz answers your Q's on the impending opening of #Obamacare insurance exchanges on C-SPAN: http://ow.ly

Kansas' Great Hope: Managed Care Will Tame Medicaid Costs http://khne.ws/12tlo3p


http://khne.ws/12tlo3p
ICYMI: Obama: 'I've Offered Sensible Reforms To Medicare' http://khne.ws/11pCIqJ


http://khne.ws/11pCIqJ
Today's headlines: CBO Predicts Bright Short Term, But Unsustainable Health Costs As Baby Boomers Retire http://khne.ws/HKMauB


http://khne.ws/HKMauB
Today's cartoon: Butcher Shop? http://khne.ws/11GEvqK


http://khne.ws/11GEvqK
Under the health law, will all plans have to cover maternity benefits in 2014? @mandrews110 answers: http://khne.ws/YAoBt9


http://khne.ws/YAoBt9
Today's headlines: 5th GOP Governor Backs Medicaid Expansion: http://khne.ws/HKMauB


http://khne.ws/HKMauB
Our @JayHancock1 will be on Baltimore radio -- NPR's WYPR -- at Noon ET today talking insurance costs: http://ow.ly/hqUSc


http://ow.ly/hqUSc
Today's cartoon: If you can't take the heat ... http://khne.ws/WMbBPa


http://khne.ws/WMbBPa
Why are some seniors switching out of private Medicare plans? @Jor

In [7]:
frame.head()

,tweetid,date,title,url
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...,http://khne.ws/1ceRuaZ
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...,http://khne.ws/1NfQjsp
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...,http://khne.ws/1Do7UrB
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...,http://khne.ws/1y8aJwC
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting...",http://khne.ws/1HUROpc


In [8]:
list_scrapt = ['//div[@class="article-body"]/p/text()',
               '',
               '//div[@class="meta"]/p/span/a/text()'
               ]

In [9]:
# Scrapt form web
frame_urls = frame['url'].values

In [10]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="article-body"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class="meta"]/p/span/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("==========================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

['ERRORS'] 404 , at http://khne.ws/1ceRuaZ
['ERRORS'] 404 , at http://khne.ws/1NfQjsp
['ERRORS'] 404 , at http://khne.ws/1Do7UrB
['ERRORS'] 404 , at http://khne.ws/1y8aJwC
['ERRORS'] 404 , at http://khne.ws/1HUROpc
['ERRORS'] 404 , at http://khne.ws/1JowAkr
['ERRORS'] 404 , at http://khne.ws/1NaiTex
['ERRORS'] 404 , at http://khne.ws/1JowAkr
Header:   200
http://cnn.it/1yVmb9H
0
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  0
tags  :  0
author  :  0



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Find all possible hastags


Header:   200
http://wamu.fm/1NQYhmG
4763
0
contents  :  4763
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/khn-video-tax-deadline-meets-the-health-law/
1462
0
contents  :  1462
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/rule-proposed-on-providing-mental-health-parity-in-medicaid-program/
4839
0
contents  :  4839
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/battle-for-mental-health-parity-produces-mixed-results/
3937
0
contents  :  3937
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/high-deductible-health-plans-can-ruin-finances/
9278
0
contents  :  9278
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/morning-breakout/billionaires-harness-money-technology-in-pursuit-of-fountain-of-youth/
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/ritual-not-science-keeps-the-annual-physical-alive/
4763
0
contents  :  4763
tags  :  

['ERRORS'] 404 , at http://khne.ws/1A44JPw
['ERRORS'] 404 , at http://khne.ws/1BOewji
['ERRORS'] 404 , at http://khne.ws/1vvrMr4
Header:   200
http://n.pr/1BHQXJ2
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/18hAZcq
['ERRORS'] 404 , at http://khne.ws/1LCdcC9
['ERRORS'] 404 , at http://khne.ws/1wh8qGM
['ERRORS'] 404 , at http://khne.ws/1AD7RWn
['ERRORS'] 404 , at http://khne.ws/1BHezgT
['ERRORS'] 404 , at http://khne.ws/1JDbXF1
Header:   200
http://Healthcare.gov
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1vlMDwX
['ERRORS'] 404 , at http://khne.ws/1zTpmOa
['ERRORS'] 404 , at http://khne.ws/1F3sKdN
['ERRORS'] 404 , at http://khne.ws/1vkGOzC
['ERRORS'] 404 , at http://khne.ws/1A9Y2Qw
['ERRORS'] 404 , at http://khne.ws/19ceN3r
['ERRORS'] 404 , at http://khne.ws/19ceN3r
Header:   200
http://kaiserhealthnews.org/toward-a-better-end-palliative-care-hospice-and-end-of-life/
1666
0
contents  :  1666
tags  :  0
author  :  0

[

['ERRORS'] 404 , at http://khne.ws/1GQmqX6
['ERRORS'] 404 , at http://khne.ws/13v9olN
['ERRORS'] 404 , at http://khne.ws/1BSz9aX
['ERRORS'] 404 , at http://khne.ws/1qX8Pez
['ERRORS'] 404 , at http://khne.ws/1zWYKQn
['ERRORS'] 404 , at http://khne.ws/134CV4N
['ERRORS'] 404 , at http://khne.ws/1wVY5jh
['ERRORS'] 404 , at http://khne.ws/1A4xvDG
['ERRORS'] 404 , at http://khne.ws/1xp9APW
['ERRORS'] 404 , at http://khne.ws/1stYBh7
['ERRORS'] 404 , at http://khne.ws/1zWYKQn
['ERRORS'] 404 , at http://khne.ws/12VewP6
['ERRORS'] 404 , at http://khne.ws/1uwYsta
['ERRORS'] 404 , at http://khne.ws/1zghTM5
['ERRORS'] 404 , at http://khne.ws/1sgE1qd
['ERRORS'] 404 , at http://khne.ws/1BAJMlg
['ERRORS'] 404 , at http://khne.ws/1AkSv7b
['ERRORS'] 404 , at http://khne.ws/1Ghx7Dy
['ERRORS'] 404 , at http://khne.ws/KHNsignup
['ERRORS'] 404 , at http://khne.ws/1x5N4vl
['ERRORS'] 404 , at http://khne.ws/1BbuaBL
['ERRORS'] 404 , at http://khne.ws/1wdWpiU
['ERRORS'] 404 , at http://khne.ws/12ufqlg
['ERRORS'

...

['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/YoHnXN
Header:   200
http://www.hhs.gov/live/index.html
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/10Md7Ds
['ERRORS'] 404 , at http://khne.ws/10TvcOH
['ERRORS'] 404 , at http://khne.ws/Zn2moO
['ERRORS'] 404 , at http://khne.ws/10TvcOH
['ERRORS'] 404 , at http://khne.ws/14brSGj
['ERRORS'] 404 , at http://khne.ws/16MytVf
['ERRORS'] 404 , at http://khne.ws/ZfKZY3
['ERRORS'] 404 , at http://khne.ws/10Rs8VQ
['ERRORS'] 404 , at http://khne.ws/YgfRMd
['ERRORS'] 404 , at http://khne.ws/Ygeltm
['ERRORS'] 404 , at http://khne.ws/12jz6qo
['ERRORS'] 404 , at http://khne.ws/12DYGXh
['ERRORS'] 404 , at http://khne.ws/10Qhi2i
['ERRORS'] 404 , at http://khne.ws/151I64g
['ERRORS'] 404 , at http://khne.ws/151I64g
['ERRORS'] 404 , at http://khne.ws/10Jygy5
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/ZwyL37
['ERRORS'] 404 , at http://khne.ws/XzAiEy
['ERRORS'] 40

['ERRORS'] 404 , at http://khne.ws/XGOiGN
['ERRORS'] 404 , at http://khne.ws/Y3jthF
['ERRORS'] 404 , at http://khne.ws/13Xk84W
['ERRORS'] 404 , at http://khne.ws/13Xk84W
['ERRORS'] 404 , at http://khne.ws/12aXIln
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/YwAv71
['ERRORS'] 404 , at http://khne.ws/YweGVk
['ERRORS'] 404 , at http://khne.ws/WgbFft
['ERRORS'] 404 , at http://khne.ws/HKMauB
Header:   200
http://www.kaiserhealthnews.org/Daily-Reports/2013/February/25/Research-Roundup.aspx
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/YSfx2s
['ERRORS'] 404 , at http://khne.ws/1266iSF
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/W7VsJe
['ERRORS'] 404 , at http://khne.ws/W7VsJe
['ERRORS'] 404 , at http://khne.ws/Znh3KJ
['ERRORS'] 404 , at http://khne.ws/YruuGD
['ERRORS'] 404 , at http://khne.ws/UR8p8H
['ERRORS'] 404 , at http://khne.ws/Znh3KJ
['ERRORS'] 404 , at http://khne.ws/Znh3KJ
['ERRORS'] 40

['ERRORS'] 404 , at http://khne.ws/SCwH54
['ERRORS'] 404 , at http://khne.ws/SCrI4o
['ERRORS'] 404 , at http://khne.ws/WmXjEp
Header:   200
http://ow.ly/h3XjH
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/WfVjAX
['ERRORS'] 404 , at http://khne.ws/14524ZG
['ERRORS'] 404 , at http://khne.ws/14524ZG
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/UkDCiR
['ERRORS'] 404 , at http://khne.ws/Wq95ib
['ERRORS'] 404 , at http://khne.ws/Vi7GhI
['ERRORS'] 404 , at http://khne.ws/WTpolX
['ERRORS'] 404 , at http://khne.ws/VnIWks
['ERRORS'] 404 , at http://khne.ws/VnIWks
['ERRORS'] 404 , at http://khne.ws/VnIWks
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/ULJi4r
Header:   200
http://n.pr/10CiRUQ
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/SeBhGw
['ERRORS'] 404 , at http://khne.ws/U6Affo
['ERRORS'] 404 , at http://khne.ws/WMD1mY
Header:   200
http://www.kaiserhealthnews.org

['ERRORS'] 404 , at http://khne.ws/VvDvh1
['ERRORS'] 404 , at http://khne.ws/VvDvh1
['ERRORS'] 404 , at http://khne.ws/UbeQOH
['ERRORS'] 404 , at http://khne.ws/RlpKpz
Header:   200
http://ow.ly/fYEFT
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/TMiAX0
['ERRORS'] 404 , at http://khne.ws/UPOq8x
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/TJYLmy
['ERRORS'] 404 , at http://khne.ws/TJWxDW
['ERRORS'] 404 , at http://khne.ws/VKABXQ
['ERRORS'] 404 , at http://khne.ws/VPh9sY
['ERRORS'] 404 , at http://khne.ws/VPh9sY
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/VKABXQ
['ERRORS'] 404 , at http://khne.ws/121dHyN
['ERRORS'] 404 , at http://khne.ws/SADgmb
['ERRORS'] 404 , at http://khne.ws/SADgmb
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/QHZOCY
['ERRORS'] 404 , at http://khne.ws/VDWc44
['ERRORS'] 404 , at http://khne.ws/WJyTVA
['ERRORS'] 404 , at http://khne.ws/Sy1dd

['ERRORS'] 404 , at http://khne.ws/VDFyQl
['ERRORS'] 404 , at http://khne.ws/HKMauB
['ERRORS'] 404 , at http://khne.ws/VE3LpY
['ERRORS'] 404 , at http://khne.ws/X15Wc3
Header:   200
http://ow.ly/eUPTe
10172
0
contents  :  10172
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/VDFyQl
['ERRORS'] 404 , at http://khne.ws/TX02YO
['ERRORS'] 404 , at http://khne.ws/Rr2AvC
['ERRORS'] 404 , at http://khne.ws/VBqp20
['ERRORS'] 404 , at http://khne.ws/VBqp20
['ERRORS'] 404 , at http://khne.ws/Yjeo4L
['ERRORS'] 404 , at http://khne.ws/W2C2oC
['ERRORS'] 404 , at http://khne.ws/VZfzsx
['ERRORS'] 404 , at http://khne.ws/PgMi8E


In [11]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [12]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,NaN,NaN,Thu Apr 09 20:11:29 +0000 2015,NaN,A new sort of extracurricular activity: seeing...,5.862602e+17,http://khne.ws/1ceRuaZ
1,NaN,NaN,Thu Apr 09 18:46:24 +0000 2015,NaN,Houston firefighters have another tool at thei...,5.862388e+17,http://khne.ws/1NfQjsp
2,NaN,NaN,Thu Apr 09 15:00:25 +0000 2015,NaN,Will #Montana expand its #Medicaid program? So...,5.861819e+17,http://khne.ws/1Do7UrB
3,NaN,NaN,Thu Apr 09 13:30:44 +0000 2015,NaN,Rand Paul's campaign first day: abortion and t...,5.861593e+17,http://khne.ws/1y8aJwC
4,NaN,NaN,Thu Apr 09 13:29:47 +0000 2015,NaN,"Enroll America, A Group Dedicated To Promoting...",5.861591e+17,http://khne.ws/1HUROpc


In [13]:
# -------------------------------------------------
frame.shape

(3505, 7)

In [14]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3505 entries, 0 to 3504
Data columns (total 7 columns):
author      233 non-null object
contents    233 non-null object
date        3505 non-null object
tags        233 non-null object
title       3505 non-null object
tweetid     3505 non-null float64
url         3505 non-null object
dtypes: float64(1), object(6)
memory usage: 379.1+ KB


In [15]:
# test_url= 'http://ow.ly/1Piv0Y'

# res = requests.get(test_url)
# if res.status_code  == 200:
# #     soup = BeautifulSoup(res.content, 'lxml')
# #     author = soup.find("span",{"class":"byline"})
# #     print(author)
# #     content = soup.find("div", {"class": "article-body"})
# #     print(content)

#     # Initialize XPath
#     xpath_selector = Selector(text=res.text)
    
#     author = xpath_selector.xpath('//*[@id="post-22962"]/header/div[2]/p/span/a/strong/text()').extract()[0]
#     print("Author: ", author)
#     print("\n")
#     content = xpath_selector.xpath('//div[@class="article-body"]/p/text()').extract()
#     content = join_txt(content)
#     print(content)
# else:
#     print("[ERROR]: ", res.status_code)

In [16]:
frame.to_csv(dest_url, sep='|', encoding='utf-8', index=False)

In [17]:
import unicodedata
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z0-9\s\']+)"
    reg_dspace = r"( {2,})"   # r"(\s{2,})"
    reg_html = r"(<[^>]+>)"
    reg_newlines = r"([\r|\n|\r\n]+)"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_html," ",contents)
            contents = unicodedata.normalize('NFKD', contents).encode('ascii', 'ignore').decode('ascii')
            contents = re.sub(reg_special," ", contents)
            contents = re.sub(reg_newlines," ",contents)
            contents = re.sub(reg_dspace," ",contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  3
[Out] key:  0
1  :  3
[Out] key:  1
2  :  3
[Out] key:  2
3  :  3
[Out] key:  3
4  :  3
[Out] key:  4
5  :  3
[Out] key:  5
6  :  3
[Out] key:  6
7  :  3
[Out] key:  7
8  :  3
[Out] key:  8
9  :  3
[Out] key:  9
10  :  4763
11  :  1462
12  :  4839
13  :  3937
14  :  9278
15  :  3
[Out] key:  15
16  :  4763
17  :  6637
18  :  4338
19  :  1462
20  :  2481
21  :  1462
22  :  3
[Out] key:  22
23  :  3
[Out] key:  23
24  :  3
[Out] key:  24
25  :  3
[Out] key:  25
26  :  3
[Out] key:  26
27  :  3
[Out] key:  27
28  :  3
[Out] key:  28
29  :  3
[Out] key:  29
30  :  3
[Out] key:  30
31  :  3
[Out] key:  31
32  :  3
[Out] key:  32
33  :  3
[Out] key:  33
34  :  3
[Out] key:  34
35  :  3
[Out] key:  35
36  :  3
[Out] key:  36
37  :  3
[Out] key:  37
38  :  3
[Out] key:  38
39  :  3
[Out] key:  39
40  :  3
[Out] key:  40
41  :  3
[Out] key:  41
42  :  3
[Out] key:  42
43  :  3
[Out] key:  43
44  :  3
[Out] key:  44
45  :  3
[Out] key:  45
46  :  3
[Out] key:  46
47  :  3
[Out] key:  47


522  :  3
[Out] key:  522
523  :  3
[Out] key:  523
524  :  3
[Out] key:  524
525  :  3
[Out] key:  525
526  :  3
[Out] key:  526
527  :  3
[Out] key:  527
528  :  3
[Out] key:  528
529  :  3
[Out] key:  529
530  :  3
[Out] key:  530
531  :  3
[Out] key:  531
532  :  3
[Out] key:  532
533  :  3
[Out] key:  533
534  :  3
[Out] key:  534
535  :  3
[Out] key:  535
536  :  3
[Out] key:  536
537  :  3
[Out] key:  537
538  :  3
[Out] key:  538
539  :  3223
540  :  6668
541  :  253
542  :  7265
543  :  3
[Out] key:  543
544  :  3
[Out] key:  544
545  :  3
[Out] key:  545
546  :  3
[Out] key:  546
547  :  3
[Out] key:  547
548  :  3
[Out] key:  548
549  :  3
[Out] key:  549
550  :  3
[Out] key:  550
551  :  3
[Out] key:  551
552  :  3
[Out] key:  552
553  :  3
[Out] key:  553
554  :  3
[Out] key:  554
555  :  3
[Out] key:  555
556  :  3
[Out] key:  556
557  :  3
[Out] key:  557
558  :  3
[Out] key:  558
559  :  3
[Out] key:  559
560  :  4361
561  :  3
[Out] key:  561
562  :  3
[Out] key:  562


1094  :  3
[Out] key:  1094
1095  :  3
[Out] key:  1095
1096  :  3
[Out] key:  1096
1097  :  3
[Out] key:  1097
1098  :  3
[Out] key:  1098
1099  :  3
[Out] key:  1099
1100  :  3
[Out] key:  1100
1101  :  3
[Out] key:  1101
1102  :  3
[Out] key:  1102
1103  :  3
[Out] key:  1103
1104  :  3
[Out] key:  1104
1105  :  3
[Out] key:  1105
1106  :  3
[Out] key:  1106
1107  :  3
[Out] key:  1107
1108  :  3
[Out] key:  1108
1109  :  3
[Out] key:  1109
1110  :  3
[Out] key:  1110
1111  :  3
[Out] key:  1111
1112  :  3
[Out] key:  1112
1113  :  3
[Out] key:  1113
1114  :  3
[Out] key:  1114
1115  :  3
[Out] key:  1115
1116  :  3
[Out] key:  1116
1117  :  3
[Out] key:  1117
1118  :  3
[Out] key:  1118
1119  :  3
[Out] key:  1119
1120  :  3
[Out] key:  1120
1121  :  3
[Out] key:  1121
1122  :  3
[Out] key:  1122
1123  :  3
[Out] key:  1123
1124  :  3
[Out] key:  1124
1125  :  3
[Out] key:  1125
1126  :  3
[Out] key:  1126
1127  :  3
[Out] key:  1127
1128  :  3
[Out] key:  1128
1129  :  3
[Out] key

1639  :  3
[Out] key:  1639
1640  :  3
[Out] key:  1640
1641  :  3
[Out] key:  1641
1642  :  3
[Out] key:  1642
1643  :  3
[Out] key:  1643
1644  :  3
[Out] key:  1644
1645  :  3
[Out] key:  1645
1646  :  3
[Out] key:  1646
1647  :  3
[Out] key:  1647
1648  :  3
[Out] key:  1648
1649  :  3
[Out] key:  1649
1650  :  3
[Out] key:  1650
1651  :  3
[Out] key:  1651
1652  :  3
[Out] key:  1652
1653  :  3
[Out] key:  1653
1654  :  3
[Out] key:  1654
1655  :  3
[Out] key:  1655
1656  :  3
[Out] key:  1656
1657  :  3
[Out] key:  1657
1658  :  3
[Out] key:  1658
1659  :  3
[Out] key:  1659
1660  :  3
[Out] key:  1660
1661  :  3
[Out] key:  1661
1662  :  3
[Out] key:  1662
1663  :  3
[Out] key:  1663
1664  :  3
[Out] key:  1664
1665  :  3
[Out] key:  1665
1666  :  3
[Out] key:  1666
1667  :  3
[Out] key:  1667
1668  :  3
[Out] key:  1668
1669  :  3
[Out] key:  1669
1670  :  3
[Out] key:  1670
1671  :  3
[Out] key:  1671
1672  :  3
[Out] key:  1672
1673  :  3
[Out] key:  1673
1674  :  3
[Out] key

2288  :  3
[Out] key:  2288
2289  :  3
[Out] key:  2289
2290  :  3
[Out] key:  2290
2291  :  3
[Out] key:  2291
2292  :  3
[Out] key:  2292
2293  :  3
[Out] key:  2293
2294  :  3
[Out] key:  2294
2295  :  3
[Out] key:  2295
2296  :  3
[Out] key:  2296
2297  :  3
[Out] key:  2297
2298  :  3
[Out] key:  2298
2299  :  3
[Out] key:  2299
2300  :  3
[Out] key:  2300
2301  :  3
[Out] key:  2301
2302  :  3
[Out] key:  2302
2303  :  3
[Out] key:  2303
2304  :  3
[Out] key:  2304
2305  :  3
[Out] key:  2305
2306  :  3
[Out] key:  2306
2307  :  3
[Out] key:  2307
2308  :  4633
2309  :  3
[Out] key:  2309
2310  :  3
[Out] key:  2310
2311  :  3
[Out] key:  2311
2312  :  3
[Out] key:  2312
2313  :  3
[Out] key:  2313
2314  :  3
[Out] key:  2314
2315  :  3
[Out] key:  2315
2316  :  3
[Out] key:  2316
2317  :  3
[Out] key:  2317
2318  :  3
[Out] key:  2318
2319  :  3
[Out] key:  2319
2320  :  3
[Out] key:  2320
2321  :  3
[Out] key:  2321
2322  :  3
[Out] key:  2322
2323  :  3
[Out] key:  2323
2324  

[Out] key:  2778
2779  :  3
[Out] key:  2779
2780  :  3
[Out] key:  2780
2781  :  3
[Out] key:  2781
2782  :  3
[Out] key:  2782
2783  :  3
[Out] key:  2783
2784  :  3
[Out] key:  2784
2785  :  3
[Out] key:  2785
2786  :  3
[Out] key:  2786
2787  :  3
[Out] key:  2787
2788  :  3
[Out] key:  2788
2789  :  3
[Out] key:  2789
2790  :  3
[Out] key:  2790
2791  :  3
[Out] key:  2791
2792  :  3
[Out] key:  2792
2793  :  3
[Out] key:  2793
2794  :  3
[Out] key:  2794
2795  :  3
[Out] key:  2795
2796  :  3
[Out] key:  2796
2797  :  3
[Out] key:  2797
2798  :  3
[Out] key:  2798
2799  :  3
[Out] key:  2799
2800  :  3
[Out] key:  2800
2801  :  3
[Out] key:  2801
2802  :  3
[Out] key:  2802
2803  :  3
[Out] key:  2803
2804  :  3
[Out] key:  2804
2805  :  3
[Out] key:  2805
2806  :  3
[Out] key:  2806
2807  :  3
[Out] key:  2807
2808  :  3
[Out] key:  2808
2809  :  3
[Out] key:  2809
2810  :  3
[Out] key:  2810
2811  :  3
[Out] key:  2811
2812  :  3
[Out] key:  2812
2813  :  3
[Out] key:  2813
281

3313  :  3
[Out] key:  3313
3314  :  3
[Out] key:  3314
3315  :  3
[Out] key:  3315
3316  :  3
[Out] key:  3316
3317  :  3
[Out] key:  3317
3318  :  3
[Out] key:  3318
3319  :  3
[Out] key:  3319
3320  :  3
[Out] key:  3320
3321  :  3
[Out] key:  3321
3322  :  3
[Out] key:  3322
3323  :  3
[Out] key:  3323
3324  :  3
[Out] key:  3324
3325  :  3
[Out] key:  3325
3326  :  3
[Out] key:  3326
3327  :  3
[Out] key:  3327
3328  :  3
[Out] key:  3328
3329  :  3
[Out] key:  3329
3330  :  3
[Out] key:  3330
3331  :  3
[Out] key:  3331
3332  :  3
[Out] key:  3332
3333  :  3
[Out] key:  3333
3334  :  3
[Out] key:  3334
3335  :  3
[Out] key:  3335
3336  :  3
[Out] key:  3336
3337  :  3
[Out] key:  3337
3338  :  3
[Out] key:  3338
3339  :  3
[Out] key:  3339
3340  :  3
[Out] key:  3340
3341  :  3
[Out] key:  3341
3342  :  3
[Out] key:  3342
3343  :  3
[Out] key:  3343
3344  :  3
[Out] key:  3344
3345  :  3
[Out] key:  3345
3346  :  3
[Out] key:  3346
3347  :  3
[Out] key:  3347
3348  :  3
[Out] key

In [18]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  30
1  :  30
2  :  30
3  :  30
4  :  30
5  :  30
6  :  30
7  :  30
8  :  30
9  :  30
10  :  30
11  :  30
12  :  30
13  :  30
14  :  30
15  :  30
16  :  30
17  :  30
18  :  30
19  :  30
20  :  30
21  :  30
22  :  30
23  :  30
24  :  30
25  :  30
26  :  30
27  :  30
28  :  30
29  :  30
30  :  30
31  :  30
32  :  30
33  :  30
34  :  30
35  :  30
36  :  30
37  :  30
38  :  30
39  :  30
40  :  30
41  :  30
42  :  30
43  :  30
44  :  30
45  :  30
46  :  30
47  :  30
48  :  30
49  :  30
50  :  30
51  :  30
52  :  30
53  :  30
54  :  30
55  :  30
56  :  30
57  :  30
58  :  30
59  :  30
60  :  30
61  :  30
62  :  30
63  :  30
64  :  30
65  :  30
66  :  30
67  :  30
68  :  30
69  :  30
70  :  30
71  :  30
72  :  30
73  :  30
74  :  30
75  :  30
76  :  30
77  :  30
78  :  30
79  :  30
80  :  30
81  :  30
82  :  30
83  :  30
84  :  30
85  :  30
86  :  30
87  :  30
88  :  30
89  :  30
90  :  30
91  :  30
92  :  30
93  :  30
94  :  30
95  :  30
96  :  30
97  :  30
98  :  30
99  :  30
100  :  30

1062  :  30
1063  :  30
1064  :  30
1065  :  30
1066  :  30
1067  :  30
1068  :  30
1069  :  30
1070  :  30
1071  :  30
1072  :  30
1073  :  30
1074  :  30
1075  :  30
1076  :  30
1077  :  30
1078  :  30
1079  :  30
1080  :  30
1081  :  30
1082  :  30
1083  :  30
1084  :  30
1085  :  30
1086  :  30
1087  :  30
1088  :  30
1089  :  30
1090  :  30
1091  :  30
1092  :  30
1093  :  30
1094  :  30
1095  :  30
1096  :  30
1097  :  30
1098  :  30
1099  :  30
1100  :  30
1101  :  30
1102  :  30
1103  :  30
1104  :  30
1105  :  30
1106  :  30
1107  :  30
1108  :  30
1109  :  30
1110  :  30
1111  :  30
1112  :  30
1113  :  30
1114  :  30
1115  :  30
1116  :  30
1117  :  30
1118  :  30
1119  :  30
1120  :  30
1121  :  30
1122  :  30
1123  :  30
1124  :  30
1125  :  30
1126  :  30
1127  :  30
1128  :  30
1129  :  30
1130  :  30
1131  :  30
1132  :  30
1133  :  30
1134  :  30
1135  :  30
1136  :  30
1137  :  30
1138  :  30
1139  :  30
1140  :  30
1141  :  30
1142  :  30
1143  :  30
1144  :  30
1145

1760  :  30
1761  :  30
1762  :  30
1763  :  30
1764  :  30
1765  :  30
1766  :  30
1767  :  30
1768  :  30
1769  :  30
1770  :  30
1771  :  30
1772  :  30
1773  :  30
1774  :  30
1775  :  30
1776  :  30
1777  :  30
1778  :  30
1779  :  30
1780  :  30
1781  :  30
1782  :  30
1783  :  30
1784  :  30
1785  :  30
1786  :  30
1787  :  30
1788  :  30
1789  :  30
1790  :  30
1791  :  30
1792  :  30
1793  :  30
1794  :  30
1795  :  30
1796  :  30
1797  :  30
1798  :  30
1799  :  30
1800  :  30
1801  :  30
1802  :  30
1803  :  30
1804  :  30
1805  :  30
1806  :  30
1807  :  30
1808  :  30
1809  :  30
1810  :  30
1811  :  30
1812  :  30
1813  :  30
1814  :  30
1815  :  30
1816  :  30
1817  :  30
1818  :  30
1819  :  30
1820  :  30
1821  :  30
1822  :  30
1823  :  30
1824  :  30
1825  :  30
1826  :  30
1827  :  30
1828  :  30
1829  :  30
1830  :  30
1831  :  30
1832  :  30
1833  :  30
1834  :  30
1835  :  30
1836  :  30
1837  :  30
1838  :  30
1839  :  30
1840  :  30
1841  :  30
1842  :  30
1843

2452  :  30
2453  :  30
2454  :  30
2455  :  30
2456  :  30
2457  :  30
2458  :  30
2459  :  30
2460  :  30
2461  :  30
2462  :  30
2463  :  30
2464  :  30
2465  :  30
2466  :  30
2467  :  30
2468  :  30
2469  :  30
2470  :  30
2471  :  30
2472  :  30
2473  :  30
2474  :  30
2475  :  30
2476  :  30
2477  :  30
2478  :  30
2479  :  30
2480  :  30
2481  :  30
2482  :  30
2483  :  30
2484  :  30
2485  :  30
2486  :  30
2487  :  30
2488  :  30
2489  :  30
2490  :  30
2491  :  30
2492  :  30
2493  :  30
2494  :  30
2495  :  30
2496  :  30
2497  :  30
2498  :  30
2499  :  30
2500  :  30
2501  :  30
2502  :  30
2503  :  30
2504  :  30
2505  :  30
2506  :  30
2507  :  30
2508  :  30
2509  :  30
2510  :  30
2511  :  30
2512  :  30
2513  :  30
2514  :  30
2515  :  30
2516  :  30
2517  :  30
2518  :  30
2519  :  30
2520  :  30
2521  :  30
2522  :  30
2523  :  30
2524  :  30
2525  :  30
2526  :  30
2527  :  30
2528  :  30
2529  :  30
2530  :  30
2531  :  30
2532  :  30
2533  :  30
2534  :  30
2535

3370  :  30
3371  :  30
3372  :  30
3373  :  30
3374  :  30
3375  :  30
3376  :  30
3377  :  30
3378  :  30
3379  :  30
3380  :  30
3381  :  30
3382  :  30
3383  :  30
3384  :  30
3385  :  30
3386  :  30
3387  :  30
3388  :  30
3389  :  30
3390  :  30
3391  :  30
3392  :  30
3393  :  30
3394  :  30
3395  :  30
3396  :  30
3397  :  30
3398  :  30
3399  :  30
3400  :  30
3401  :  30
3402  :  30
3403  :  30
3404  :  30
3405  :  30
3406  :  30
3407  :  30
3408  :  30
3409  :  30
3410  :  30
3411  :  30
3412  :  30
3413  :  30
3414  :  30
3415  :  30
3416  :  30
3417  :  30
3418  :  30
3419  :  30
3420  :  30
3421  :  30
3422  :  30
3423  :  30
3424  :  30
3425  :  30
3426  :  30
3427  :  30
3428  :  30
3429  :  30
3430  :  30
3431  :  30
3432  :  30
3433  :  30
3434  :  30
3435  :  30
3436  :  30
3437  :  30
3438  :  30
3439  :  30
3440  :  30
3441  :  30
3442  :  30
3443  :  30
3444  :  30
3445  :  30
3446  :  30
3447  :  30
3448  :  30
3449  :  30
3450  :  30
3451  :  30
3452  :  30
3453